In [3]:
import pandas as pd
import matplotlib as plt
import rasterio
import numpy as np

Crime data loading

In [4]:
df = pd.read_csv("C:/Users/User/PycharmProjects/SafeRoute/NYPD_Arrests_Data__Historic_.csv") # TODO change to relative path
df = df.replace('(null)', np.nan)
df = df.replace('nan', np.nan)
df = df.dropna(axis=0)
df

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
1,237354740,12/04/2021,153.0,RAPE 3,104.0,RAPE,PL 1302502,F,B,41,0.0,25-44,M,WHITE HISPANIC,1013232.0,236725.0,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034)
2,236081433,11/09/2021,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,Q,113,0.0,25-44,M,BLACK,1046367.0,186986.0,40.679700,-73.776047,POINT (-73.77604736799998 40.67970040800003)
4,192799737,01/26/2019,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,0.0,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)
7,236106641,11/10/2021,263.0,"ARSON 2,3,4",114.0,ARSON,PL 1501001,F,B,41,72.0,25-44,M,WHITE HISPANIC,1017934.0,232221.0,40.804013,-73.878332,POINT (-73.87833183299993 40.804012949000025)
8,238383628,12/28/2021,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1702500,F,Q,113,0.0,18-24,M,BLACK,1045482.0,191341.0,40.691660,-73.779199,POINT (-73.77919852099996 40.69166001700007)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5498645,255851599,12/13/2022,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO",881.0,OTHER TRAFFIC INFRACTION,VTL051101A,M,M,32,0.0,25-44,M,BLACK,1000188.0,238820.0,40.822167,-73.942415,POINT (-73.94241483 40.82216691)
5498646,256469262,12/16/2022,198.0,CRIMINAL CONTEMPT 1,126.0,MISCELLANEOUS PENAL LAW,PL 21551B3,F,B,46,0.0,25-44,M,BLACK,1011750.0,250274.0,40.853578,-73.900591,POINT (-73.900591 40.853578)
5498647,260305039,12/20/2022,548.0,"GAMBLING, DEVICE, POSSESSION",350.0,GAMBLING,PL 2253001,M,K,72,0.0,25-44,F,ASIAN / PACIFIC ISLANDER,981944.0,171869.0,40.638420,-74.008307,POINT (-74.008307 40.63842)
5498648,255843041,12/13/2022,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,5,0.0,45-64,M,ASIAN / PACIFIC ISLANDER,986159.0,200104.0,40.715917,-73.993111,POINT (-73.993111 40.715917)


ALL OFFENCE DESCRIPTION

In [5]:
# np.unique(np.array())
list(set(df['OFNS_DESC']))

['ADMINISTRATIVE CODES',
 'GAMBLING',
 'UNAUTHORIZED USE OF A VEHICLE',
 'MISCELLANEOUS PENAL LAW',
 'OTHER STATE LAWS (NON PENAL LAW)',
 'DANGEROUS WEAPONS',
 'FORTUNE TELLING',
 'UNDER THE INFLUENCE, DRUGS',
 'POSSESSION OF STOLEN PROPERTY 5',
 'OFFENSES INVOLVING FRAUD',
 'CRIMINAL MISCHIEF & RELATED OF',
 'ENDAN WELFARE INCOMP',
 'FELONY ASSAULT',
 'MURDER & NON-NEGL. MANSLAUGHTER',
 'ANTICIPATORY OFFENSES',
 'OFFENSES AGAINST MARRIAGE UNCLASSIFIED',
 'JOSTLING',
 'OTHER OFFENSES RELATED TO THEF',
 'NYS LAWS-UNCLASSIFIED FELONY',
 'FORGERY',
 'DISORDERLY CONDUCT',
 'HARASSMENT',
 'CHILD ABANDONMENT/NON SUPPORT',
 'LOITERING FOR DRUG PURPOSES',
 'UNLAWFUL POSS. WEAP. ON SCHOOL',
 'OFFENSES RELATED TO CHILDREN',
 'VEHICLE AND TRAFFIC LAWS',
 'OTHER OFFENSES RELATED TO THEFT',
 'THEFT-FRAUD',
 'CANNABIS RELATED OFFENSES',
 'LOITERING',
 'FELONY SEX CRIMES',
 'POSSESSION OF STOLEN PROPERTY',
 'LOITERING/GAMBLING (CARDS, DIC',
 'OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV',
 'OFFENSES A

ASSIGNING ARBITRARY DANGER VALUES FOR CRIME

In [6]:
danger_scores = {
    'HOMICIDE-NEGLIGENT,UNCLASSIFIED': 10,
    'SEX CRIMES': 9,
    'GRAND LARCENY': 3,
    'ROBBERY': 8,
    'UNLAWFUL POSS. WEAP. ON SCHOOL GROUNDS': 6,
    'KIDNAPPING & RELATED OFFENSES': 9,
    'BURGLARY': 6,
    'DANGEROUS DRUGS': 5,
    'FRAUDS': 2,
    'RAPE': 12,
    'FELONY SEX CRIMES': 10,
    'ASSAULT 3 & RELATED OFFENSES': 8,
    'UNAUTHORIZED USE OF A VEHICLE 3 (UUV)': 4,
    'FELONY ASSAULT': 9,
    'MURDER & NON-NEGL. MANSLAUGHTER': 15,
    'THEFT-FRAUD': 3,
    'PETIT LARCENY': 2
}

FORMING NEW DATAFRAME

In [7]:
new_df = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), ['ARREST_DATE', 'Latitude', 'Longitude']]
new_df

,ARREST_DATE,Latitude,Longitude
1,12/04/2021,40.816392,-73.895296
2,11/09/2021,40.679700,-73.776047
4,01/26/2019,40.800694,-73.941109
9,01/06/2016,40.648650,-73.950336
10,12/04/2021,40.688584,-73.916526
...,...,...,...
5498636,12/11/2022,40.637301,-74.009474
5498639,12/30/2022,40.671104,-73.913562
5498641,12/13/2022,40.742954,-73.955274
5498648,12/13/2022,40.715917,-73.993111


In [8]:
new_df['Danger Score'] = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), 'OFNS_DESC'].map(danger_scores)
new_df

,ARREST_DATE,Latitude,Longitude,Danger Score
1,12/04/2021,40.816392,-73.895296,12
2,11/09/2021,40.679700,-73.776047,9
4,01/26/2019,40.800694,-73.941109,9
9,01/06/2016,40.648650,-73.950336,12
10,12/04/2021,40.688584,-73.916526,8
...,...,...,...,...
5498636,12/11/2022,40.637301,-74.009474,5
5498639,12/30/2022,40.671104,-73.913562,8
5498641,12/13/2022,40.742954,-73.955274,9
5498648,12/13/2022,40.715917,-73.993111,8


In [9]:
new_df.sort_values(by=['Longitude'])

,ARREST_DATE,Latitude,Longitude,Danger Score
3031914,03/27/2012,40.507709,-74.254939,5
3015760,03/27/2012,40.507709,-74.254939,5
2948228,06/05/2012,40.508303,-74.254560,8
3287428,08/02/2011,40.508303,-74.254560,9
3291149,07/20/2011,40.508303,-74.254560,5
...,...,...,...,...
3836331,04/23/2010,51.627063,-73.681780,8
3283455,07/30/2011,51.627063,-73.681780,8
3965562,12/11/2009,51.627063,-73.681780,5
4226739,05/22/2009,51.627063,-73.681780,5


In [10]:
(new_df['Latitude'].min(), new_df['Latitude'].max(), new_df['Longitude'].min(), new_df['Longitude'].max())

(40.49890536300006, 62.08307497800007, -74.25493873599999, -73.68178026799995)

OPENING CLUSTERED GEOTIF POPULATION FILE

In [11]:
with rasterio.open("C:/Users/User/PycharmProjects/SafeRoute/clustered.tif") as src:
    data = src.read()
    image = src.read(1)
    clusters = src.read(2)
    bounds = src.bounds
    

In [12]:
bounds

BoundingBox(left=-74.27999999999999, bottom=40.400000004, right=-73.66000000247999, top=41.4)

In [13]:
(abs(bounds[2] - bounds[0]), abs(bounds[3] - bounds[1]))

(0.6199999975199972, 0.9999999959999997)

In [14]:
step = (abs(bounds[2] - bounds[0])/np.shape(clusters)[1] + abs(bounds[3] - bounds[1])/np.shape(clusters)[0]) / 2
step

0.000833333329999998

COUNTING POPULATION PER CLUSTER

In [15]:
pop_dict = {c: [] for c in np.unique(clusters)}
pop_count = {c: 0 for c in np.unique(clusters)}
for iy, ix in np.ndindex(image.shape):
    ap = image[iy, ix] if image[iy, ix] > 0 else 0
    pop_dict[clusters[iy, ix]].append(ap)
for k, v in pop_dict.items():
    pop_count[k] = sum(pop_dict[k])/len(pop_dict[k]) if len(pop_dict[k]) > 0 else 0
pop_count

{-1: 20.838352248063266,
 0: 0.0,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 403.9698924731183,
 22: 374.0883116883117,
 23: 344.7806788511749,
 24: 322.2092050209205,
 25: 310.781512605042,
 26: 284.07209612817087,
 27: 256.8208695652174,
 28: 243.7421052631579,
 29: 229.93288590604027,
 30: 220.3644859813084,
 31: 212.32441471571906,
 32: 204.45980707395498,
 33: 189.22164948453607,
 34: 178.60687432867883,
 35: 153.88075880758808,
 36: 166.32436974789917,
 37: 158.967845659164,
 38: 137.83928571428572,
 39: 131.9090909090909,
 40: 119.16029143897997,
 41: 113.46315789473684,
 42: 109.69086651053864,
 43: 101.84454756380511,
 44: 83.06192660550458,
 45: 59.8028545941124,
 46: 76.45161290322581,
 47: 73.1283185840708,
 48: 68.96498905908096,
 49: 55.930487137383686,
 50: 63.03768115942029,
 51: 54.53503184713376,
 52: 51.9157303370786

In [16]:
cluster_score = {c: 0 for c in np.unique(clusters)}
cluster_crime_num = {c: 0 for c in np.unique(clusters)}
cluster_score

{-1: 0,
 0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0,
 117: 0,
 118: 0,
 119: 0,
 120: 0,
 121: 0,
 122

CALCULATING CRIME NUMBER AND SCORE PER CLUSTER

In [17]:
for row in new_df.itertuples():
    p = (row.Longitude, row.Latitude) # point declaration
    if abs(p[0])>abs(bounds[0]) or abs(p[0])<abs(bounds[2]): 
        continue
    elif abs(p[1])<abs(bounds[1]) or abs(p[1])>abs(bounds[3]): 
        continue
    cluster_score[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += row._4
    cluster_crime_num[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += 1

In [18]:
cluster_score

{-1: 12977720,
 0: 0,
 1: 0,
 2: 16,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 7345,
 9: 16934,
 10: 0,
 11: 1512,
 12: 40508,
 13: 250806,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 59692,
 19: 0,
 20: 321450,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 186,
 62: 0,
 63: 676,
 64: 0,
 65: 683,
 66: 0,
 67: 2347,
 68: 0,
 69: 0,
 70: 0,
 71: 67,
 72: 133,
 73: 14447,
 74: 13080,
 75: 0,
 76: 0,
 77: 28439,
 78: 0,
 79: 0,
 80: 0,
 81: 76111,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 46411,
 87: 0,
 88: 79270,
 89: 90562,
 90: 0,
 91: 68455,
 92: 46107,
 93: 0,
 94: 117966,
 95: 118335,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 851,
 101: 161456,
 102: 0,
 103: 876,
 104: 20123,
 105: 22124,
 106: 4444,
 107: 359821,
 108: 597,


In [19]:
cluster_score[-1] = 0
cluster_crime_num[-1] = 0
cluster_score

{-1: 0,
 0: 0,
 1: 0,
 2: 16,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 7345,
 9: 16934,
 10: 0,
 11: 1512,
 12: 40508,
 13: 250806,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 59692,
 19: 0,
 20: 321450,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 186,
 62: 0,
 63: 676,
 64: 0,
 65: 683,
 66: 0,
 67: 2347,
 68: 0,
 69: 0,
 70: 0,
 71: 67,
 72: 133,
 73: 14447,
 74: 13080,
 75: 0,
 76: 0,
 77: 28439,
 78: 0,
 79: 0,
 80: 0,
 81: 76111,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 46411,
 87: 0,
 88: 79270,
 89: 90562,
 90: 0,
 91: 68455,
 92: 46107,
 93: 0,
 94: 117966,
 95: 118335,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 851,
 101: 161456,
 102: 0,
 103: 876,
 104: 20123,
 105: 22124,
 106: 4444,
 107: 359821,
 108: 597,
 109: 6

FINDING AREA PER CLUSTER

In [20]:
cluster_area = {c: (np.count_nonzero(clusters == c))*100 for c in np.unique(clusters)}
cluster_area

{-1: 49529300,
 0: 104600,
 1: 38100,
 2: 40800,
 3: 54600,
 4: 28900,
 5: 31600,
 6: 26500,
 7: 78300,
 8: 41600,
 9: 62800,
 10: 64000,
 11: 51900,
 12: 45700,
 13: 20500,
 14: 96600,
 15: 2131500,
 16: 20000,
 17: 278800,
 18: 49000,
 19: 26600,
 20: 16145900,
 21: 46500,
 22: 38500,
 23: 38300,
 24: 23900,
 25: 35700,
 26: 74900,
 27: 57500,
 28: 38000,
 29: 44700,
 30: 42800,
 31: 29900,
 32: 31100,
 33: 38800,
 34: 93100,
 35: 36900,
 36: 59500,
 37: 31100,
 38: 39200,
 39: 18700,
 40: 54900,
 41: 19000,
 42: 42700,
 43: 43100,
 44: 87200,
 45: 112100,
 46: 15500,
 47: 22600,
 48: 45700,
 49: 182700,
 50: 34500,
 51: 15700,
 52: 17800,
 53: 27300,
 54: 37800,
 55: 15800,
 56: 63300,
 57: 16800,
 58: 21400,
 59: 36000,
 60: 31200,
 61: 53600,
 62: 16600,
 63: 18400,
 64: 21000,
 65: 18900,
 66: 25100,
 67: 40700,
 68: 33200,
 69: 550900,
 70: 15600,
 71: 25200,
 72: 15400,
 73: 29200,
 74: 36700,
 75: 46900,
 76: 32300,
 77: 16000,
 78: 21100,
 79: 22100,
 80: 24800,
 81: 42400,
 

In [21]:
for k, v in cluster_score.items():
    cluster_score[k] = v/cluster_area[k]
    cluster_crime_num[k] = cluster_crime_num[k]/cluster_area[k]

cluster_score

{-1: 0.0,
 0: 0.0,
 1: 0.0,
 2: 0.000392156862745098,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.1765625,
 9: 0.2696496815286624,
 10: 0.0,
 11: 0.029132947976878612,
 12: 0.8863894967177243,
 13: 12.234439024390245,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 1.2182040816326531,
 19: 0.0,
 20: 0.01990907908509281,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0034701492537313433,
 62: 0.0,
 63: 0.03673913043478261,
 64: 0.0,
 65: 0.036137566137566135,
 66: 0.0,
 67: 0.05766584766584767,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0026587301587301586,
 72: 0.008636363636363636,
 73: 0.49476027397260275,
 74: 0.3564032697547684,
 

In [22]:
fin_df = pd.DataFrame({"pop_count_adj": pop_count ,'crime_number_adj': cluster_crime_num, 'crime_score_adj': cluster_score})
fin_df.to_csv('crime_score.csv', index_label='cluster')
fin_df

,pop_count_adj,crime_number_adj,crime_score_adj
-1,20.838352,0.000000,0.000000
0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000
2,0.000000,0.000049,0.000392
3,0.000000,0.000000,0.000000
...,...,...,...
242,0.000000,0.005070,0.031246
243,0.000000,0.000000,0.000000
244,0.000000,0.000000,0.000000
245,0.000000,0.000000,0.000000


MERGING DATAFRAMES FOR FINAL FILE

In [23]:
df1 = pd.read_csv('crime_score.csv')
df2 = pd.read_csv('average_income.csv')
merged_df = pd.merge(df1, df2, on='cluster', how='outer')
merged_df.to_csv('all_data.csv', index=False)


ADJUSTING INCOME

In [28]:
edited_df = merged_df[(merged_df != 0).all(axis=1)]
i_l = edited_df['income'].tolist()

In [29]:
v_min = min([value for value in i_l if value >= 1000])
v_max = max([value for value in i_l if value >= 1000])
for i in range(len(i_l)):
    i_l[i] = ((i_l[i] - v_min) if v_min < i_l[i] else 0)
    i_l[i] = i_l[i]/v_max

In [30]:
edited_df['income'] = i_l
edited_df.to_csv('edited_data.csv', index=False)

C:\Users\User\AppData\Local\Temp\ipykernel_5232\1900860770.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edited_df['income'] = i_l


In [31]:
df666 = pd.read_csv('edited_data.csv')
df666

,cluster,pop_count_adj,crime_number_adj,crime_score_adj,income
0,61,34.164179,0.000560,0.003470,0.508690
1,63,31.000000,0.005815,0.036739,0.000000
2,65,29.994709,0.005820,0.036138,0.032664
3,67,26.867322,0.010000,0.057666,0.078692
4,71,23.702381,0.000357,0.002659,0.389814
5,72,22.318182,0.001429,0.008636,0.110012
6,73,20.410959,0.083938,0.494760,0.118248
7,74,18.833787,0.059292,0.356403,0.358639
8,81,15.320755,0.311297,1.795071,0.382542
9,86,14.000000,0.513434,2.795843,0.314803
